# Creating a Vector DB

Before creating our vector search, we need to create a database of vectors. Notebook will show you how to do this:

In [1]:
## Create iris cursor
## I've hidden the code for this in the file Utils.get_iris_connection - its very simple and shown in the previous tutorial
from Utils.get_iris_connection import get_cursor
cursor = get_cursor()

### Download data and load into pandas dataframe

In [2]:
sql = """SELECT 
DocumentReferenceContentAttachmentData, DocumentReferenceSubjectReference
FROM VectorSearchApp.DocumentReference"""

cursor.execute(sql)
out = cursor.fetchall()

import pandas as pd
## The result_set doesn't include column names so we will add them ourselvs
cols = ["ClinicalNotes", "Patient"] 

df = pd.DataFrame(out, columns=cols)
df["PatientID"] = pd.to_numeric(df["Patient"].astype(str).str.strip("Patient/"))

df.head()

,ClinicalNotes,Patient,PatientID
0,4f7469746973204d65646961204576616c756174696f6e...,Patient/3,3
1,446174653a20323032352d30382d30360a50726f766964...,Patient/3,3
2,466f6c6c6f772d557020666f72204f7469746973204d65...,Patient/3,3
3,446174653a20323032342d31312d32310a50726f766964...,Patient/3,3
4,446174653a20323032342d30382d30360a50726f766964...,Patient/3,3


### Decode the clinical notes into plain text

In [3]:
import base64

## here I am using a python lambda function (single-line function) to take a row of data 
## and decode the ClinicalNotes column, and return it to a new column called NotesDecoded 
df["NotesDecoded"] = df["ClinicalNotes"].apply(lambda x: bytes.fromhex(x).decode("utf-8", errors="replace"))
df.head()

,ClinicalNotes,Patient,PatientID,NotesDecoded
0,4f7469746973204d65646961204576616c756174696f6e...,Patient/3,3,Otitis Media Evaluation\nDate: 2024-01-25\nPro...
1,446174653a20323032352d30382d30360a50726f766964...,Patient/3,3,Date: 2025-08-06\nProvider: Dr. Jason347 Frami...
2,466f6c6c6f772d557020666f72204f7469746973204d65...,Patient/3,3,Follow-Up for Otitis Media\nDate: 2024-05-07\n...
3,446174653a20323032342d31312d32310a50726f766964...,Patient/3,3,Date: 2024-11-21\nProvider: Dr. Chin306 Kulas5...
4,446174653a20323032342d30382d30360a50726f766964...,Patient/3,3,Date: 2024-08-06\nProvider: Dr. Jason347 Frami...


In [4]:
## View one example of a note: 
print(df["NotesDecoded"][0])

Otitis Media Evaluation
Date: 2024-01-25
Provider: Dr. Lemuel304 Stokes453
Location: Beth Israel Deaconess Hospital – Needham
Reason for Visit: Ear pain and irritability
Subjective:
Aurora presented with symptoms of ear discomfort, mild fever, and increased fussiness. Parent reports onset 2 days ago. No vomiting or diarrhea. No prior history of ear infections.
Objective:

Vitals: Temp 37.8°C, BP 107/80 mmHg
Physical Exam:

Tympanic membrane: Erythematous and bulging on the right side
No discharge noted
Mild tenderness on palpation of the mastoid process
Lungs clear, no respiratory distress



Assessment:

Acute Otitis Media – Right ear

Plan:

Prescribed Amoxicillin 250 mg oral capsule, 1 capsule twice daily for 7 days
Supportive care: fluids, rest, acetaminophen for fever
Follow-up in 10 days or sooner if symptoms worsen
Educated parent on signs of complications


### Encoding clinical notes to vectors

Here we are using a pre-trained model from the hugging-faces `sentance_transformers` libary. I have done this using the model `all-MiniLM-L6-v2`. This is a basic sentence transformer model which outputs vectors with 384 dimensions. There are many models available, some of which will be more relevant to your needs. For example, if you are using a language other than english, you would likely need a multi-lingual model or a model in your specific language. 

In [5]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence transformer model. This model's output vectors are of size 384

model = SentenceTransformer('all-MiniLM-L6-v2') 

# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df['NotesDecoded'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['Notes_Vector'] = embeddings.tolist()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

As we are using health data, an example of a better model for this process would be [pritamdeka/S-PubMedBert-MS-MARCO](https://huggingface.co/pritamdeka/S-PubMedBert-MS-MARCO), which is designed for medical text. This model has double the number of dimensions (768) as the model used above, which will slow the time taken to create the vectors. For this reason, I'm going to stick with the smaller model for now. 

If you do change the model used, remember to change the number of dimensions given when defining the SQL table below. 

In [6]:
## View output
df.head()

,ClinicalNotes,Patient,PatientID,NotesDecoded,Notes_Vector
0,4f7469746973204d65646961204576616c756174696f6e...,Patient/3,3,Otitis Media Evaluation\nDate: 2024-01-25\nPro...,"[0.037052784115076065, -0.01652533747255802, 0..."
1,446174653a20323032352d30382d30360a50726f766964...,Patient/3,3,Date: 2025-08-06\nProvider: Dr. Jason347 Frami...,"[-0.03362147510051727, 0.05015568435192108, 0...."
2,466f6c6c6f772d557020666f72204f7469746973204d65...,Patient/3,3,Follow-Up for Otitis Media\nDate: 2024-05-07\n...,"[-0.0073646255768835545, -0.02232077717781067,..."
3,446174653a20323032342d31312d32310a50726f766964...,Patient/3,3,Date: 2024-11-21\nProvider: Dr. Chin306 Kulas5...,"[-0.02497003600001335, 0.051254600286483765, 0..."
4,446174653a20323032342d30382d30360a50726f766964...,Patient/3,3,Date: 2024-08-06\nProvider: Dr. Jason347 Frami...,"[-0.006600924767553806, 0.04176628217101097, 0..."


### Create new table in IRIS

Here we create a SQL query to create a new table in IRIS.

We have to define the datatypes required for this table. The vectors are loaded in the IRIS-specific data type 'VECTOR'. We also specify the type of each element in the vector (Double - a floating point number) and the dimensionality of the vector (384).


In [7]:
table_name = "VectorSearch.DocRefVectors"

create_table_query = f"""
CREATE TABLE {table_name} (
PatientID INTEGER,
ClinicalNotes LONGVARCHAR,
NotesVector VECTOR(DOUBLE, 384)
)
"""

In [8]:
cursor.execute(create_table_query)

0

### Adding data

We will now insert our vector dataset into the table we've generated. There are multiple ways of adding data to our table, here I will show two and do a very brief speed comparison.

Firstly we insert each row individually, iterating over each row individually. For this iteration, we are using df.apply() to efficiently perform the same function to each row of the data table. 

Secondly, we use `cursor.executemany()` with a single query and a list of parameter lists to execute all the insertions at once. 

Importantly, the IRIS-SQL `TO_VECTOR()` function needs the vector to be in string format, so you will see both methods involve converting this data to a string before executing the query.,

In [9]:
## Create a reusable query string with ? placeholders for the values
insert_query = f"INSERT INTO {table_name} ( PatientID, ClinicalNotes, NotesVector) values (?, ?, TO_VECTOR(?))"

In [10]:
## Needed for a speed comparison
import time

In [11]:
## Method 1: Inserting rows one at time (iterating with df.apply())
st = time.time()

def addRow(row): ## Create a function to insert each row
    cursor.execute(insert_query, [ row["PatientID"], row["NotesDecoded"], str(row["Notes_Vector"])])
## Apply the row insertion function to each row in the table (axis=1 specifies that we are iterating over rows, not columns). 
df.apply(addRow, axis=1)

print(f"Method 1 took {time.time()-st} Seconds")

Method 1 took 0.17098617553710938 Seconds


In [12]:
## Reset the table between the insertion methods
cursor.execute(f"Drop TABLE {table_name}" )
cursor.execute(create_table_query)

0

In [13]:
st = time.time()
df["Notes_Vector_str"] = df["Notes_Vector"].astype(str)
rows_list = df[["PatientID", "NotesDecoded", "Notes_Vector_str"]].values.tolist()

cursor.executemany(insert_query, rows_list)
print(f"Method 2 took {time.time()-st} Seconds")

Method 2 took 0.05177664756774902 Seconds


Both methods are quick because our table is very small. However, the second method is generally quicker. This speed boast might be useful if you are dealing with very large datasets. Either way though, we now have our data table complete with Vectors for use in a vector search! 


### Querying table

Before moving on to the vector search, lets quickly query the database to check everything looks as expected:

In [14]:
sql_query  = f"Select TOP 3 * FROM {table_name}"
cursor.execute(sql_query) 
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns= ["PatientID","NotesDecoded", "Notes_Vector"]  )
results_df.head()

,PatientID,NotesDecoded,Notes_Vector
0,3,Otitis Media Evaluation\nDate: 2024-01-25\nPro...,".037052784115076065063,-.016525337472558021546..."
1,3,Date: 2025-08-06\nProvider: Dr. Jason347 Frami...,"-.033621475100517272949,.050155684351921081542..."
2,3,Follow-Up for Otitis Media\nDate: 2024-05-07\n...,"-.0073646255768835544586,-.0223207771778106689..."
